# Synovium, treatment naive sc -- Process Pseudobulks + Bulks


# Imports

In [1]:
import sys
from buddi import buddi
from buddi.preprocessing import sc_preprocess


# general imports
import warnings
import numpy as np
import os
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
import collections
import scanpy as sc
import anndata as ad


# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import sklearn as sk

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import pickle
import gzip
from pathlib import Path


/Users/davidnat/Documents/projects/greenelab/buddi_test/.venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/davidnat/Documents/projects/greenelab/buddi_test/.venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/davidnat/

# Parameters

In [2]:
# parameters

aug_data_path = f"{os.getcwd()}/../../data/single_cell_data/augmented_synovium_data/"
cibersort_path = f"{os.getcwd()}/../..//data/single_cell_data/cibersort_synovium/"
data_path = f"{os.getcwd()}/../../data/single_cell_data/synovium/"



#####################
### set the study ###
#####################

res_name = "all-synovium"
in_name = "synovium_processed"
processed_sc_file = f"{data_path}/{in_name}.h5ad"



# Load and Process data

### Read in data and metadata

In [3]:
# read in the data

adata = sc.read_h5ad(processed_sc_file)

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`



In [4]:

# get the columns we need to iterate over for making pseudobulks
adata.obs['scpred_CellType'] = adata.obs['cell_type'].tolist()


adata.obs['sample_id'] = adata.obs['sample'].tolist()
adata.obs['stim'] = "CTRL"

# make the gene_ids col
adata.var['gene_ids'] = adata.var.index.tolist()

# we use everything as training
adata.obs["isTraining"] = "Train"

### look at some data stats

In [5]:
# each sample should only have cells in with "STIM" or "CTRL"
tab = adata.obs.groupby(['sample_id', 'stim']).size()
tab.unstack()

stim,CTRL
sample_id,
BRI-415,7370
BRI-421,9031
BRI-436,5655
BRI-458,9327
BRI-460,9103
BRI-462,10648
BRI-515,7944
BRI-566,9638
BRI-601,8244


In [6]:
# see how many cells per cell type
adata.obs["scpred_CellType"].value_counts()


Myeloid        22220
Fibroblast     22182
T_CD4          17059
B              11300
T_CD8           8768
Endothelial     3689
T_other         3285
NK              2472
Name: scpred_CellType, dtype: int64

### write out data to use in CIBERSORTx
we use the signature genes from CIBERSORTx as one of our inputs to BuDDI


In [7]:
# write out data for cibersort

adata_dense = adata[np.where(adata.obs.sample_id == "BRI-415")[0]]
dense_matrix = adata_dense.X.todense()


sc_profile_file = os.path.join(aug_data_path, f"{res_name}_sig.pkl")
sc_profile_path = Path(sc_profile_file)

dense_df = pd.DataFrame(dense_matrix, columns = adata_dense.var['gene_ids'])
dense_df.insert(loc=0, column='scpred_CellType', value=adata_dense.obs["scpred_CellType"].to_list())


pickle.dump( dense_df, open( sc_profile_path, "wb" ) )


# Make pseudobulks

In [8]:
# write out the gene ids
gene_pass = adata.var['gene_ids']
gene_out_file = os.path.join(aug_data_path, f"{res_name}_genes.pkl")
gene_out_path = Path(gene_out_file)
pickle.dump( gene_pass, open( gene_out_path, "wb" ) )

# metadata
sample_order = ['BRI-462', 'BRI-566', 'BRI-458', 'BRI-460',
                'BRI-421', 'BRI-601', 'BRI-605', 'BRI-515', 
                'BRI-415', 'BRI-623', 'BRI-436']
stim_order = ['CTRL']
train_order = ['Train']

# now generate all the proportions
total_meta_df = pd.DataFrame(columns = ["sample_id", "stim", "isTraining"])

# no cell noise 
len_vector = adata.obs["scpred_CellType"].unique().shape[0]
cell_noise = [np.random.lognormal(0, 0, adata.var['gene_ids'].shape[0]) for i in range(len_vector)]

# cell type order
cell_order = adata.obs.scpred_CellType.unique()

# simulate different number of cells
num_cells = 200 # 5000
idx = 0
for curr_samp in sample_order:
  for curr_stim in stim_order:
      for curr_train in train_order:

        print(f"running {curr_samp} {curr_stim} {curr_train}")


        # make the pseudobulks
        subset_idx = np.logical_and(adata.obs.sample_id == curr_samp, adata.obs.stim == curr_stim)
        subset_idx = np.where(np.logical_and(subset_idx, adata.obs.isTraining == curr_train))[0]
        if len(subset_idx) == 0:
            continue
        
        temp_adata = adata[subset_idx]

        print("make_prop_and_sum")
        prop_df, pseudobulks_df, test_prop_df, test_pseudobulks_df = sc_preprocess.make_prop_and_sum(temp_adata, 
                                                                                num_samples=5000, #1000
                                                                                num_cells=num_cells,
                                                                                use_true_prop=False,
                                                                                cell_noise=cell_noise,
                                                                                useSampleNoise=False)
        # number of random pseudobulks
        num_rand_pseudo = pseudobulks_df.shape[0] 

        # get the single cell type proportions
        print("get_single_celltype_prop_matrix")
        ct_prop_df = sc_preprocess.get_single_celltype_prop_matrix(num_samp=100, # 100
                                                                    cell_order=cell_order)

        # now get the cell-type specific pseudobulks
        print("use_prop_make_sum")
        prop_df_sc, pseudobulks_df_sc, _ = sc_preprocess.use_prop_make_sum(temp_adata,  
                                                                            num_cells=num_cells, 
                                                                            props_vec=ct_prop_df, 
                                                                            cell_noise=cell_noise,
                                                                            sample_noise=None,
                                                                            useSampleNoise=False)
        # number of random pseudobulks
        num_ct_pseudo = pseudobulks_df_sc.shape[0] 


        # put them together
        print("concat")        
        prop_df = pd.concat([prop_df,prop_df_sc])
        pseudobulks_df = pd.concat([pseudobulks_df, pseudobulks_df_sc])

        # make the metadata
        num_samps = pseudobulks_df.shape[0] 
        samp_type = ["bulk"]*num_samps
        cell_prop_type = ["random"]*num_rand_pseudo+["cell_type_specific"]*num_ct_pseudo 
        samp_type = ["sc_ref"]*(num_rand_pseudo+num_ct_pseudo)
        
        metadata_df = pd.DataFrame(data = {"sample_id":[curr_samp]*num_samps, 
                                          "stim":[curr_stim]*num_samps,
                                          "isTraining":[curr_train]*num_samps,
                                          "cell_prop_type":cell_prop_type,
                                          "samp_type":samp_type,})

        # make the proportions instead of cell counts
        prop_df = prop_df.div(prop_df.sum(axis=1), axis=0)
        pseudobulk_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_pseudo_splits.pkl")
        prop_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_prop_splits.pkl")
        meta_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_meta_splits.pkl")

        print("write")        
        pseudobulk_path = Path(pseudobulk_file)
        prop_path = Path(prop_file)
        meta_path = Path(meta_file)
        pickle.dump( prop_df, open( prop_path, "wb" ) )
        pickle.dump( pseudobulks_df, open( pseudobulk_path, "wb" ) )
        pickle.dump( metadata_df, open( meta_path, "wb" ) )





running BRI-462 CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
300
400
500
600
700
concat
write
running BRI-566 CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
300
400
500
600
700
concat
write
running BRI-458 CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
